# CBS

In [27]:
from os import listdir
import numpy as np

In [ ]:
def read_map_from_movingai_file(path):
    map_file = open(path)
    count = 0
    name = map_file.readline()
    height = int(map_file.readline().split()[1])
    width = int(map_file.readline().split()[1])
    type_ = map_file.readline()
    cells = [[0 for _ in range(width)] for _ in range(height)]
    
    i = 0
    j = 0

    for l in map_file:
        j = 0
        for c in l:
            if c == '.':
                cells[i][j] = 0
            elif c == 'T' or c == '@':
                cells[i][j] = 1
            else:
                continue
            
            j += 1
            
        if j != width:
            raise Exception("Size Error. Map width = ", j, ", but must be", width, "(map line: ", i, ")")
                
        i += 1
        if(i == height):
            break
    
    return (width, height, cells)

In [31]:
def read_tasks_from_movingai_file(path):
    tasks = []
    task_file = open(path)
    for l in task_file:
        new_task = l.split()[4:]
        if len(new_task) != 0:
            tasks.append(list(map(float,new_task)))
    #возвращает числа: размеры карты, координаты начала и конца, длину пути
    return np.array(tasks)

In [38]:
task = read_tasks_from_movingai_file("scen-even/Berlin_1_256-even-1.scen")
task[:,4].sum()

180448.62036054

In [23]:
def massive_test(search_function, num_agents, *args):
    stat = dict()
    taskMap = Map()
    
    stat["corr"] = []
    stat["len"] = []
    stat["nc"] = []
    stat["st"] = []
    
    #считываем карты
    map_file_names = ["cbs/arena0.map","cbs/arena1.map","cbs/arena2.map"]
    
    #для каждой карты у меня есть папка заданий
    task_file_folders = ["cbs/scen0","cbs/scen1","cbs/scen1"]
    
    
    for num in range(3):
        nodes_created = 0
        num_steps = 0
        correct_true = 0
        correct_false = 0
        error_factor = 0
        
        width, height, cells = read_map_from_movingai_file(map_file_names[num])
        taskMap = Map()
        taskMap.set_grid_cells(width,height,cells)
        
        #считываем файлы из нужной папки (в файле конфигурация начальная)
        taskfiles = [f for f in listdir(task_file_folders[num])]       
        
        for taskCount in range(len(taskfiles)):
            try:
                #теперь тут уже 2Д массив для К агентов
                task = read_tasks_from_movingai_file(taskfiles[taskCount])
                iStarts = task[:num_agents,0].astype('int')
                jStarts = task[:num_agents,1].astype('int')
                iGoals = task[:num_agents,2].astype('int')
                jGoals = task[:num_agents,3].astype('int')
                
                tot_length = task[:num_agents,4].sum()

                result = search_function(taskMap, iStarts, jStarts, iGoals, jGoals, *args)

                # хрень какая-то
                nodesExpanded = result[2]
                nodesOpened = result[3]
                
                # нашли решение
                if result[0]:
                    # закидываем сюда все найденные пути, в result[1] должен храниться массив 
                    # концевых вершин каждого агента
                    paths = make_paths(result[1]) 
                    # make_paths будет возвращать в первом аргументе массив путей, а во втором их суммарную длину
                    stat["len"].append(paths[1])
                    correct = int(path[1])  == int(length)
                    
                    correct_true += correct
                    correct_false += not correct
                    nodes_created += len(nodesOpened) + len(nodesExpanded)
                    num_steps += len(nodesExpanded)
                    error_factor += (int(path[1]) / int(length) - 1)*100
                    
                    stat["corr"].append(correct)
                    
                else:
                    print("Path not found!")
                    stat["corr"].append(False)
                    stat["len"].append(0.0)

                stat["nc"].append(len(nodesOpened) + len(nodesExpanded))
                stat["st"].append(len(nodesExpanded))

            except Exception as e:
                error = error
                print("Execution error")
                print(e)
        print("Всего путей", correct_true + correct_false, 
              ". Из них правильных: ", correct_true,
              " Неправильных: ", correct_false, 
              "Среднее число созданных узлов: ", nodes_created / (correct_true + correct_false), 
              "Среднее число шагов: ", num_steps / (correct_true + correct_false), 
              "Средняя относительная ошибка, % : ", error_factor / (correct_true + correct_false))
        
        print("------------------------NEW MAP-------------------------")

    return stat